The model only has copying mechanism. Train and validate it on respective datasets.

The model will be saved in model_output directory in the colab session. Make sure to donload the weights, vocabulary, and config info, because after the session end they will be lost

The model is evaluated on test set and the predicions will be saved in predict_out directory. Make sure to save them as well

The BLEU score with current parameters on valid and test set is 0.19 approximately. Which is almost same as Corec ugmented model results.



Mount drive to Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install the required dependencies

In [ ]:
!pip install allennlp-models
!pip install --upgrade google-cloud-storage
!!pip install allennlp

Create a directory in runtime files where the trained model will be saved. (PLease note that once the runtime is disconected the files will be lost)

In [ ]:
!mkdir model_output

Define the model hyperparameters and dataset reader and train the model
Training on GPU takes about 1h and 34min

Paths to the training and testing datasets is specified:
"train_data_path": "/content/drive/My Drive/combined_data/train.tsv",
"validation_data_path": "/content/drive/My Drive/combined_data/valid.tsv",
Please provide your own path instead

In [ ]:
from allennlp_models.generation.dataset_readers.copynet_seq2seq import CopyNetDatasetReader
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer
import tempfile
import json
from allennlp_models.generation.dataset_readers import copynet_seq2seq
from allennlp.commands.train import train_model
from allennlp.common.params import Params


config = {
  "dataset_reader": {
    "source_tokenizer": {
        "split_on_spaces":True,
        },
    "target_namespace": "target_tokens",
    "type": "copynet_seq2seq",
    "source_token_indexers": {
      "tokens": {
        "type": "single_id",
        "namespace": "source_tokens"
      }
    }
  },
  "vocabulary": {
  "min_count": {
    "source_tokens": 5,
    "target_tokens": 5,   
  },
  "oov_token":'',
},
"train_data_path": "/content/drive/My Drive/combined_data/train.tsv",
"validation_data_path": "/content/drive/My Drive/combined_data/valid.tsv",
"model": {
  "type": "copynet_seq2seq",
  "source_embedder": {
      "token_embedders": {"tokens": {"type": "embedding", "embedding_dim": 512, "trainable": True, "vocab_namespace": "source_tokens"},               
      },

  },
  "encoder": {
    "type": "lstm",
    "input_size": 512,
    "hidden_size": 512,
    "num_layers": 2,
    "dropout": 0.1,
    "bidirectional": True
  },
  "attention": {
    "type": "bilinear",
    "vector_dim": 1024,
    "matrix_dim": 1024,
  },
  # "target_embedding_dim": 100,
  "beam_size": 5,
  "max_decoding_steps": 50,
  #'scheduled_sampling_ratio':0.5,
},
"data_loader": {

  "batch_size" : 32,
  "shuffle":True,
},
"trainer": {
  "optimizer": {
    "type": "adam",
    "lr": 0.001
  },
  "learning_rate_scheduler": {
    "type": "cosine",
    "t_initial": 5,
    "eta_mul": 0.9
  },
  "checkpointer": {
      "keep_most_recent_by_count": 17,
    },
  "num_epochs": 17,
  "cuda_device": 0,
  "should_log_learning_rate": True,
  "should_log_parameter_statistics": False
}
}



serialization_dir = "model_output"
config_filename = serialization_dir + "/training_config.json"
with open(config_filename, "w") as config_file:
    json.dump(config, config_file)


model = train_model(
    Params(config), serialization_dir, file_friendly_logging=True, force=True
)


Use the saved pre-trained model weigths to create a new model and test on the test dataset

In [ ]:
from allennlp.models.model import Model
from allennlp.common.params import Params
from allennlp_models.generation.dataset_readers.copynet_seq2seq import CopyNetDatasetReader
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer
import tempfile
import json
from allennlp_models.generation.dataset_readers import copynet_seq2seq

# it's necessary to specify the configuration again
config = {
  "dataset_reader": {
    "source_tokenizer": {
        "split_on_spaces":True,
        },
    "target_namespace": "target_tokens",
    "type": "copynet_seq2seq",
    "source_token_indexers": {
      "tokens": {
        "type": "single_id",
        "namespace": "source_tokens"
      }
    }
  },
  "vocabulary": {
  "min_count": {
    "source_tokens": 5,
    "target_tokens": 5,   
  },
  "oov_token":'',
},
"model": {
  "type": "copynet_seq2seq",
  "source_embedder": {
      "token_embedders": {"tokens": {"type": "embedding", "embedding_dim": 512, "trainable": True, "vocab_namespace": "source_tokens"},               
      },

  },
  "encoder": {
    "type": "lstm",
    "input_size": 512,
    "hidden_size": 512,
    "num_layers": 2,
    "dropout": 0.1,
    "bidirectional": True
  },
  "attention": {
    "type": "bilinear",
    "vector_dim": 1024,
    "matrix_dim": 1024,
  },
  # "target_embedding_dim": 100,
  "beam_size": 5,
  "max_decoding_steps": 50,
  #'scheduled_sampling_ratio':0.5,
},
"data_loader": {

  "batch_size" : 32,
  "shuffle":True,
},
"trainer": {
  "optimizer": {
    "type": "adam",
    "lr": 0.001
  },
  "learning_rate_scheduler": {
    "type": "cosine",
    "t_initial": 5,
    "eta_mul": 0.9
  },
  "checkpointer": {
      "keep_most_recent_by_count": 17,
    },
  "num_epochs": 17,
  "cuda_device": 0,
  "should_log_learning_rate": True,
  "should_log_parameter_statistics": False
}
}

model = Model.load(Params(config),serialization_dir='model_output', weights_file="model_output/model_state_e14_b0.th")

In [ ]:
!mkdir predict_out

In [ ]:
# predictor
from allennlp.data.dataset_readers import dataset_reader
from allennlp_models.generation.dataset_readers.copynet_seq2seq import CopyNetDatasetReader
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.data_loaders.simple_data_loader import SimpleDataLoader
from allennlp.training.util import evaluate
from allennlp.data import DatasetReader

reader = CopyNetDatasetReader(
    target_namespace="target_tokens",
    source_token_indexers={'tokens': SingleIdTokenIndexer(namespace="source_tokens")},
    )

from allennlp.predictors import Predictor

predictor = Predictor(model.to('cuda'), reader)

inputs = {
    "sentence": "mmm a / CHANGELOG . md <nl> ppp b / CHANGELOG . md <nl> # Changelog <nl> - # 2 . 2 . 0 ( 16 / 07 / 2015 ) - SNAPSHOT <nl> + # 2 . 1 . 1 ( 29 / 02 / 2016 ) - SNAPSHOT <nl> - Added AppCompat Styles"
}

from allennlp.data import Instance

print(Instance({"sentence": "this is me"}))

predictor.predict_instance(Instance({"sentence": "this is me"}))

In [ ]:
from allennlp.data.dataset_readers import dataset_reader
from allennlp_models.generation.dataset_readers.copynet_seq2seq import CopyNetDatasetReader
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.data_loaders.simple_data_loader import SimpleDataLoader
from allennlp.training.util import evaluate
from allennlp.data import DatasetReader

reader = CopyNetDatasetReader(
    target_namespace="target_tokens",
    source_token_indexers={'tokens': SingleIdTokenIndexer(namespace="source_tokens")},
    )

test_data = list(reader.read("/content/drive/My Drive/combined_data/test_example.tsv"))
data_loader = SimpleDataLoader(test_data, batch_size=1)
data_loader.index_with(model.vocab)

In [ ]:
import time
start_time = time.time()
results = evaluate(model, data_loader)
#results = evaluate(model, data_loader, predictions_output_file='predict_out/predictions.tsv')
print("--- %s seconds ---" % (time.time() - start_time))
print(results)

In [ ]:
{'BLEU': 0.18798859277510044, 'loss': 30.710134963917582}